##### Copyright 2019 The TensorFlow Authors.

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [0]:
#@title MIT License
#
# Copyright (c) 2017 François Chollet
#
# Permission is hereby granted, free of charge, to any person obtaining a
# copy of this software and associated documentation files (the "Software"),
# to deal in the Software without restriction, including without limitation
# the rights to use, copy, modify, merge, publish, distribute, sublicense,
# and/or sell copies of the Software, and to permit persons to whom the
# Software is furnished to do so, subject to the following conditions:
#
# The above copyright notice and this permission notice shall be included in
# all copies or substantial portions of the Software.
#
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL
# THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING
# FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER
# DEALINGS IN THE SOFTWARE.

# 保存和恢复模型 

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://tensorflow.google.cn/tutorials/keras/save_and_load"><img src="https://tensorflow.google.cn/images/tf_logo_32px.png" />在 tensorflow.google.cn 上查看</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/zh-cn/tutorials/keras/save_and_load.ipynb"><img src="https://tensorflow.google.cn/images/colab_logo_32px.png" />在 Google Colab 运行</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/zh-cn/tutorials/keras/save_and_load.ipynb"><img src="https://tensorflow.google.cn/images/GitHub-Mark-32px.png" />在 Github 上查看源代码</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/zh-cn/tutorials/keras/save_and_load.ipynb"><img src="https://tensorflow.google.cn/images/download_logo_32px.png" />下载此 notebook</a>
  </td>
</table>

Note: 我们的 TensorFlow 社区翻译了这些文档。因为社区翻译是尽力而为， 所以无法保证它们是最准确的，并且反映了最新的
[官方英文文档](https://www.tensorflow.org/?hl=en)。如果您有改进此翻译的建议， 请提交 pull request 到
[tensorflow/docs](https://github.com/tensorflow/docs) GitHub 仓库。要志愿地撰写或者审核译文，请加入
[docs-zh-cn@tensorflow.org Google Group](https://groups.google.com/a/tensorflow.org/forum/#!forum/docs-zh-cn)。

模型可以在训练期间和训练完成后进行保存。这意味着模型可以从任意中断中恢复，并避免耗费比较长的时间在训练上。保存也意味着您可以共享您的模型，而其他人可以通过您的模型来重新创建工作。在发布研究模型和技术时，大多数机器学习从业者分享：

* 用于创建模型的代码
* 模型训练的权重 (weight) 和参数 (parameters) 。

共享数据有助于其他人了解模型的工作原理，并使用新数据自行尝试。

注意：小心不受信任的代码——Tensorflow 模型是代码。有关详细信息，请参阅 [安全使用Tensorflow](https://github.com/tensorflow/tensorflow/blob/master/SECURITY.md)。

### 选项

保存 Tensorflow 的模型有许多方法——具体取决于您使用的 API。本指南使用 [tf.keras](https://tensorflow.google.cn/guide/keras)， 一个高级 API 用于在 Tensorflow 中构建和训练模型。有关其他方法的实现，请参阅 TensorFlow  [保存和恢复](https://tensorflow.google.cn/guide/saved_model)指南或[保存到 eager](https://tensorflow.google.cn/guide/eager#object-based_saving)。

## 配置

### 安装并导入

安装并导入Tensorflow和依赖项：

In [0]:
try:
  # Colab only
  %tensorflow_version 2.x
except Exception:
    pass

!pip install pyyaml h5py  # 需要以 HDF5 格式保存模型

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

import os

import tensorflow as tf
from tensorflow import keras

print(tf.version.VERSION)

### 获取示例数据集

要演示如何保存和加载权重，您将使用 [MNIST 数据集](http://yann.lecun.com/exdb/mnist/). 要加快运行速度，请使用前1000个示例：

In [0]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

### 定义模型

首先构建一个简单的序列（sequential）模型：

In [0]:
# 定义一个简单的序列模型
def create_model():
  model = tf.keras.models.Sequential([
    keras.layers.Dense(512, activation='relu', input_shape=(784,)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10, activation='softmax')
  ])

  model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

  return model

# 创建一个基本的模型实例
model = create_model()

# 显示模型的结构
model.summary()

## 在训练期间保存模型（以 checkpoints 形式保存）

您可以使用训练好的模型而无需从头开始重新训练，或在您打断的地方开始训练，以防止训练过程没有保存。 `tf.keras.callbacks.ModelCheckpoint` 允许在训练的*过程中*和*结束时*回调保存的模型。

### Checkpoint 回调用法

创建一个只在训练期间保存权重的 `tf.keras.callbacks.ModelCheckpoint` 回调：

In [0]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# 创建一个保存模型权重的回调
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

# 使用新的回调训练模型
model.fit(train_images, 
          train_labels,  
          epochs=10,
          validation_data=(test_images,test_labels),
          callbacks=[cp_callback])  # 通过回调训练

# 这可能会生成与保存优化程序状态相关的警告。
# 这些警告（以及整个笔记本中的类似警告）是防止过时使用，可以忽略。

这将创建一个 TensorFlow checkpoint 文件集合，这些文件在每个 epoch 结束时更新：

In [0]:
!ls {checkpoint_dir}

创建一个新的未经训练的模型。仅恢复模型的权重时，必须具有与原始模型具有相同网络结构的模型。由于模型具有相同的结构，您可以共享权重，尽管它是模型的不同*实例*。
现在重建一个新的未经训练的模型，并在测试集上进行评估。未经训练的模型将在机会水平（chance levels）上执行（准确度约为10％）：

In [0]:
# 创建一个基本模型实例
model = create_model()

# 评估模型
loss, acc = model.evaluate(test_images,  test_labels, verbose=2)
print("Untrained model, accuracy: {:5.2f}%".format(100*acc))

然后从 checkpoint 加载权重并重新评估：

In [0]:
# 加载权重
model.load_weights(checkpoint_path)

# 重新评估模型
loss,acc = model.evaluate(test_images,  test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

### checkpoint 回调选项

回调提供了几个选项，为 checkpoint 提供唯一名称并调整 checkpoint 频率。

训练一个新模型，每五个 epochs 保存一次唯一命名的 checkpoint ：

In [0]:
# 在文件名中包含 epoch (使用 `str.format`)
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# 创建一个回调，每 5 个 epochs 保存模型的权重
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    period=5)

# 创建一个新的模型实例
model = create_model()

# 使用 `checkpoint_path` 格式保存权重
model.save_weights(checkpoint_path.format(epoch=0))

# 使用新的回调*训练*模型
model.fit(train_images, 
              train_labels,
              epochs=50, 
              callbacks=[cp_callback],
              validation_data=(test_images,test_labels),
              verbose=0)

现在查看生成的 checkpoint 并选择最新的 checkpoint ：

In [0]:
! ls {checkpoint_dir}

In [0]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

注意: 默认的 tensorflow 格式仅保存最近的5个 checkpoint 。

如果要进行测试，请重置模型并加载最新的 checkpoint ：

In [0]:
# 创建一个新的模型实例
model = create_model()

# 加载以前保存的权重
model.load_weights(latest)

# 重新评估模型
loss, acc = model.evaluate(test_images,  test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

## 这些文件是什么？

上述代码将权重存储到 [checkpoint](https://tensorflow.google.cn/guide/saved_model#save_and_restore_variables)—— 格式化文件的集合中，这些文件仅包含二进制格式的训练权重。 Checkpoints 包含：
* 一个或多个包含模型权重的分片。
* 索引文件，指示哪些权重存储在哪个分片中。

如果你只在一台机器上训练一个模型，你将有一个带有后缀的碎片： `.data-00000-of-00001`

## 手动保存权重

您将了解如何将权重加载到模型中。使用 `Model.save_weights` 方法手动保存它们同样简单。默认情况下， `tf.keras` 和 `save_weights` 特别使用 TensorFlow [checkpoints](../../guide/keras/checkpoints) 格式 `.ckpt` 扩展名和 ( 保存在 [HDF5](https://js.tensorflow.org/tutorials/import-keras.html) 扩展名为 `.h5`  [保存并序列化模型](../../guide/keras/save_and_serialize#weights-only_saving_in_savedmodel_format) )：

In [0]:
# 保存权重
model.save_weights('./checkpoints/my_checkpoint')

# 创建模型实例
model = create_model()

# Restore the weights
model.load_weights('./checkpoints/my_checkpoint')

# Evaluate the model
loss,acc = model.evaluate(test_images,  test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

## 保存整个模型

模型和优化器可以保存到包含其状态（权重和变量）和模型参数的文件中。这可以让您导出模型，以便在不访问原始 python 代码的情况下使用它。而且您可以通过恢复优化器状态的方式，从中断的位置恢复训练。

保存完整模型会非常有用——您可以在 TensorFlow.js ([HDF5](https://js.tensorflow.org/tutorials/import-keras.html), [Saved Model](https://js.tensorflow.org/tutorials/import-saved-model.html)) 加载他们，然后在 web 浏览器中训练和运行它们，或者使用  TensorFlow Lite 将它们转换为在移动设备上运行([HDF5](https://tensorflow.google.cn/lite/convert/python_api#exporting_a_tfkeras_file_), [Saved Model](https://tensorflow.google.cn/lite/convert/python_api#exporting_a_savedmodel_))

### 将模型保存为HDF5文件

Keras 可以使用 [HDF5](https://en.wikipedia.org/wiki/Hierarchical_Data_Format) 标准提供基本保存格式。出于我们的目的，可以将保存的模型视为单个二进制blob：

In [0]:
# 创建一个新的模型实例
model = create_model()

# 训练模型
model.fit(train_images, train_labels, epochs=5)

# 将整个模型保存为HDF5文件
model.save('my_model.h5')

现在，从该文件重新创建模型：

In [0]:
# 重新创建完全相同的模型，包括其权重和优化程序
new_model = keras.models.load_model('my_model.h5')

# 显示网络结构
new_model.summary()

检查其准确率（accuracy）：

In [0]:
loss, acc = new_model.evaluate(test_images,  test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

这项技术可以保存一切:

* 权重
* 模型配置(结构)
* 优化器配置

Keras 通过检查网络结构来保存模型。目前，它无法保存 Tensorflow 优化器（调用自 `tf.train`）。使用这些的时候，您需要在加载后重新编译模型，否则您将失去优化器的状态。


### 通过 `saved_model` 保存

注意：这种保存 `tf.keras` 模型的方法是实验性的，在将来的版本中可能有所改变。

建立一个新模型，然后训练它：

In [0]:
model = create_model()

model.fit(train_images, train_labels, epochs=5)

创建一个 `saved_model`，并将其放在带有 `tf.keras.experimental.export_saved_model` 的带时间戳的目录中：

In [0]:
import time
saved_model_path = "./saved_models/{}".format(int(time.time()))

tf.keras.experimental.export_saved_model(model, saved_model_path)
saved_model_path

列出您保存的模型：

In [0]:
!ls saved_models/

从保存的模型重新加载新的 Keras 模型：

In [0]:
new_model = tf.keras.experimental.load_from_saved_model(saved_model_path)

# 显示网络结构
new_model.summary()

使用恢复的模型运行预测：

In [0]:
model.predict(test_images).shape

In [0]:
# 必须在评估之前编译模型。
# 如果仅部署已保存的模型，则不需要此步骤。

new_model.compile(optimizer=model.optimizer,  # 保留已加载的优化程序
                  loss='sparse_categorical_crossentropy', 
                  metrics=['accuracy'])

# 评估已恢复的模型
loss, acc = new_model.evaluate(test_images,  test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))